In [1]:
# Importing required libraries

import pandas as pd
from datetime import datetime
import re
import glob



# Defining the input file header

header=['PatientID', 'PatientMRN', 'LastName', 'FirstName', 'MiddleName', 'DateofBirth', 'Sex', 'Metric_Desc', 'Metric_Abbr', 'Result', 'UnitOfMeasure', 'CreateDTTM', 'PerformedDTTM', 'DecodedValue', 'QOCClassificationDE', 'AnswerDE', 'VitalValue', 'NormalizedValue', 'NumericFinding', 'Unit_Abbr', 'RecordedDTTM', 'QODE', 'Vital_Status', 'Mode_Name', 'ActivityHeaderID', 'ItemID', 'LoincLabCode', 'APM_Patient_ID', 'APM_MRN', 'APM_Patient_First_Name', 'APM_Patient_Last_Name', 'APM_Patient_MI', 'APM_Patient_Street1', 'APM_Patient_Street2', 'APM_Patient_City', 'APM_Patient_State', 'APM_Patient_Zip_Code', 'APM_Patient_Country', 'APM_Patient_Sex', 'APM_Patient_DOB', 'Patient_Home_Phone', 'Patient_Work_Phone', 'Patient_Work_Ext', 'Patient_Cell_Phone', 'Patient_Primary_Phone_Type', 'Patient_Primary_Phone_Number', 'Patient_Email']
# Read the Input File and convert to pandas dataframe
input_files = glob.glob('EPIC/Input.txt')
df_input = pd.read_csv(input_files[0], delimiter='|', header=None, names=header)
df_input.fillna('', inplace=True)
pd.set_option('display.max_columns', None)




# Read the Input File and convert to pandas dataframe

input_files = glob.glob('EPIC/Vitals_allscript.csv')
Vitals_allscript_df = pd.read_csv(input_files[0], delimiter=',',)
pd.set_option('display.max_columns', None)
# Vitals_allscript_df.head(5)
# df_input.columns.tolist()



# Apply strptime to each element of the 'CreateDTTM' column
df_input['reformated_CreateDTTM'] = pd.to_datetime(df_input['CreateDTTM'], format='%Y-%m-%d %H:%M:%S.%f').dt.strftime('%m-%d-%Y %H:%M')

# Convert 'reformated_CreateDTTM' back to datetime objects
df_input['reformated_CreateDTTM'] = pd.to_datetime(df_input['reformated_CreateDTTM'], format='%m-%d-%Y %H:%M')




df_input.head()
df_input['DateOfBirth'] = df_input['DateofBirth'].str.replace('-', '', regex=False)
df_input['CreateDTTM_converted'] = df_input['CreateDTTM'].str.replace('-', '', regex=False).str.replace(' ', '', regex=False).str.replace(':', '', regex=False).str.replace('.', '', regex=False).str[:-3]



lookup_table = {

    #MSH Segment
    "MSH-1": {"Source Column": None, "Default Value": None},
    "MSH-2": {"Source Column": None, "Default Value": None},
    "MSH-3": {"Source Column": None, "Default Value": "TouchWorks"},
    "MSH-4": {"Source Column": None, "Default Value": "MountainWest"},
    "MSH-5": {"Source Column": None, "Default Value": "Rhapsody^Rhapsody"},
    "MSH-6": {"Source Column": None, "Default Value": "Epic^Epic"},
    "MSH-7": {"Source Column": None, "Default Value": datetime.now().strftime("%Y%m%d%H%M%S")},
    "MSH-8": {"Source Column": None, "Default Value": None},
    "MSH-9": {"Source Column": None, "Default Value": "ORU^R01"},
    "MSH-10": {"Source Column": None, "Default Value": None},
    "MSH-11": {"Source Column": None, "Default Value": "P"},
    "MSH-12": {"Source Column": None, "Default Value": "2.5.1"},



    #PID segment

    "PID-1": {"Source Column": None, "Default Value": None},
    "PID-2": {"Source Column": None, "Default Value": None},
    "PID-3.1": {"Source Column": "PatientMRN", "Default Value": None},
    "PID-3.2": {"Source Column": None, "Default Value": None},
    "PID-3.3": {"Source Column": None, "Default Value": None},
    "PID-3.4": {"Source Column": None, "Default Value": None},
    "PID-3.5": {"Source Column": None, "Default Value": "TWSMAMRN"},
    "PID-4": {"Source Column": None, "Default Value": None},
    "PID-5.1": {"Source Column": "LastName", "Default Value": None},
    "PID-5.2": {"Source Column": "FirstName", "Default Value": None},
    "PID-5.3": {"Source Column": "MiddleName", "Default Value": None},
    "PID-6": {"Source Column": None, "Default Value": None},
    "PID-7": {"Source Column": "DateOfBirth", "Default Value": None},
    "PID-8": {"Source Column": "Sex", "Default Value": None},
    "PID-9": {"Source Column": None, "Default Value": None},
    "PID-10": {"Source Column": None, "Default Value": None},
    "PID-11": {"Source Column": None, "Default Value": None},
    "PID-12": {"Source Column": None, "Default Value": None},
    "PID-13": {"Source Column": None, "Default Value": None},
    "PID-14": {"Source Column": None, "Default Value": None},
    "PID-15": {"Source Column": None, "Default Value": None},
    "PID-16": {"Source Column": None, "Default Value": None},
    "PID-17": {"Source Column": None, "Default Value": None},
    "PID-18": {"Source Column": None, "Default Value": None},


    #OBR segment

    "OBR-1": {"Source Column": None, "Default Value": None},
    "OBR-2": {"Source Column": None, "Default Value": None},
    "OBR-3": {"Source Column": None, "Default Value": None},
    "OBR-4": {"Source Column": None, "Default Value": None},
    "OBR-5": {"Source Column": None, "Default Value": None},
    "OBR-6": {"Source Column": None, "Default Value": None},
    "OBR-7": {"Source Column": "RecordedDTTM", "Default Value": None},


    #OBX segment

    "OBX-1": {"Source Column": None, "Default Value": None},
    "OBX-2": {"Source Column": None, "Default Value": None},
    "OBX-3": {"Source Column": "Metric_Abbr", "Default Value": None},
    "OBX-4":{"Source Column":None,"Default Value":None},
    "OBX-5": {"Source Column": "Result", "Default Value": None},
    "OBX-6": {"Source Column": "Unit_Abbr", "Default Value": None},
    "OBX-7":{"Source Column":None,"Default Value":None},
    "OBX-8": {"Source Column": None, "Default Value": None},
    "OBX-9":{"Source Column":None,"Default Value":None},
    "OBX-10":{"Source Column":None,"Default Value":None},
    "OBX-11": {"Source Column":None, "Default Value": None},
    "OBX-12":{"Source Column":None,"Default Value":None},
    "OBX-13":{"Source Column":None,"Default Value":None},
    "OBX-14":{"Source Column":"?","Default Value":None}
}

def convert_to_int(value):

    try:
        return int(float(value))
    except (ValueError, TypeError):
        return value

pd.set_option('display.max_rows', None)

def get_value(field, row):

      source_column = lookup_table[field]["Source Column"]
      default_value = lookup_table[field]["Default Value"]

      if source_column:
          if isinstance(source_column, list):
              return "_".join([str(row.get(col, '')) for col in source_column])
          return row.get(source_column, default_value)
      return default_value

def parse_to_hl7(df_input, lookup_table):

    hl7_lines = []
    obx_counter = {}
    obr_counter = {}
    empty_list = []
    count = 0

    for index, row in df_input.iterrows():

      PatientMRN= row['PatientMRN']
      if PatientMRN not in obr_counter:
            obr_counter[PatientMRN] = 1
      else:
            obr_counter[PatientMRN] += 1

      #Create MSH segment

      msh_segment = [
          f"MSH|^~\&|{get_value('MSH-3', row)}|{get_value('MSH-4', row)}|{get_value('MSH-5', row)}|{get_value('MSH-6', row)}|{get_value('MSH-7', row)}||{get_value('MSH-9', row)}|{get_value('MSH-10', row)}|{get_value('MSH-11', row)}|{get_value('MSH-12', row)}"

              ]

      # Create PID segment
      pid_segment = [
                  f"PID|||{get_value('PID-3.1', row)}^^^^{get_value('PID-3.5', row)}||{get_value('PID-5.1', row)}^{get_value('PID-5.2', row)}||{get_value('PID-7', row)}|{get_value('PID-8', row)}"

                  ]
      # Create OBR segment
      obr_segment = [

                  f"OBR|{obr_counter[PatientMRN]}||||||{get_value('OBR-3', row)}|||||||||{convert_to_int(get_value('OBR-7', row))}"

              ]
        
      obx_segment = [
            f"OBX|{obx_counter[RecordedDTTM]}||{row_get('LoincLabCode', row)}^{row_get('Metric_Abbr', row)}^LN||{row_get('Result', row)}|{row_get('Unit_Abbr', row)}"
        ]

      hl7_message = "\n".join(msh_segment + pid_segment + obr_segment + obx_segment)

      hl7_lines.append(hl7_message)

    return hl7_lines
# Parse the input DataFrame to HL7 formatted output

hl7_output = parse_to_hl7(df_input, lookup_table)
# hl7_output


def sanitize_filename(filename):
    sanitized = re.sub(r'[<>:"/\\|?*]', '_', filename)
    sanitized = sanitized.replace(' ', '_')
    return sanitized


def remove_duplicate_segments(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    unique_segments = {'MSH': False, 'PID': False, 'ORC': False, 'OBR': False}
    with open(file_path, 'w') as file:
        for line in lines:
            segment_type = line.split('|')[0]
            if segment_type in unique_segments:
                if not unique_segments[segment_type]:
                    file.write(line)
                    unique_segments[segment_type] = True




file_paths = []
for index, row in df_input.iterrows():
    filename = f"{row['PatientMRN']}"
    sanitized_filename = sanitize_filename(filename)
    file_path = f"EPIC/{sanitized_filename}.hl7"
    file_paths.append(file_path)
    with open(file_path, 'w') as file:
        for line in hl7_output:
                file.write(line + "\n")

for file_path in set(file_paths):
    remove_duplicate_segments(file_path)



print(hl7_output)

<>:161: SyntaxWarning: invalid escape sequence '\&'
<>:161: SyntaxWarning: invalid escape sequence '\&'
/var/folders/pc/6rmkz3b536l1k66dbrhqbl2r0000gn/T/ipykernel_33095/3605166121.py:161: SyntaxWarning: invalid escape sequence '\&'
  f"MSH|^~\&|{get_value('MSH-3', row)}|{get_value('MSH-4', row)}|{get_value('MSH-5', row)}|{get_value('MSH-6', row)}|{get_value('MSH-7', row)}||{get_value('MSH-9', row)}|{get_value('MSH-10', row)}|{get_value('MSH-11', row)}|{get_value('MSH-12', row)}"
/var/folders/pc/6rmkz3b536l1k66dbrhqbl2r0000gn/T/ipykernel_33095/3605166121.py:161: SyntaxWarning: invalid escape sequence '\&'
  f"MSH|^~\&|{get_value('MSH-3', row)}|{get_value('MSH-4', row)}|{get_value('MSH-5', row)}|{get_value('MSH-6', row)}|{get_value('MSH-7', row)}||{get_value('MSH-9', row)}|{get_value('MSH-10', row)}|{get_value('MSH-11', row)}|{get_value('MSH-12', row)}"


IndexError: list index out of range

In [7]:
import csv

# Paths for input and output files
header_file_path = 'header.txt'  # Path to header.txt
input_file_path = 'Input.txt'  # Path to Input.txt
output_csv_path = 'output.csv'  # Path to save the generated CSV

# Read headers from the header file
with open(header_file_path, 'r') as header_file:
    headers = header_file.readline().strip().split('|')

# Read the input file and write it to CSV format
with open(input_file_path, 'r') as input_file, open(output_csv_path, 'w', newline='') as output_csv_file:
    writer = csv.writer(output_csv_file)

    # Write headers to the CSV
    writer.writerow(headers)

    # Iterate through each row in the input file and write it to the CSV
    for line in input_file:
        row = line.strip().split('|')
        writer.writerow(row)

print(f"CSV file has been generated and saved to {output_csv_path}")


CSV file has been generated and saved to output.csv


In [9]:
import pandas as pd

# Load the vitals csv
vitals_csv_path = 'Vitals_allscript.csv'  # Update with your actual file path
vitals_df = pd.read_csv(vitals_csv_path)

# Print column names to inspect them
print(vitals_df.columns)


Index(['Source Column/Field name', 'Source Example', 'Description', 'Format',
       'HL7 field', 'Required/Highly recommended/Nice to have/',
       'Default/Generic value', 'HL7 example', 'Notes', 'Follow up'],
      dtype='object')


In [11]:
import pandas as pd
from datetime import datetime

# Load the CSV files
output_csv_path = 'output.csv'  # Path to your output CSV
vitals_csv_path = 'Vitals_allscript.csv'  # Path to your vitals description CSV

# Read the CSV files
output_df = pd.read_csv(output_csv_path)
vitals_df = pd.read_csv(vitals_csv_path)

# Print column names to ensure correct access
print("Output CSV Columns:", output_df.columns)
print("Vitals Description CSV Columns:", vitals_df.columns)

# Convert datetime to HL7 format (YYYYMMDDHHMMSS)
def convert_datetime(dt_str):
    try:
        return datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S.%f').strftime('%Y%m%d%H%M%S')
    except:
        return dt_str

# Function to create HL7-like text from data
def create_hl7_message(output_row, vitals_df):
    hl7_message = []
    
    # Create MSH (Message Header) Segment
    msh_segment = f'MSH|^~\\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|{convert_datetime(output_row["CreateDTTM"])}||ORU^R01|{output_row["PatientId"]}_{output_row["ActivityHeaderID"]}_{convert_datetime(output_row["PerformedDTTM"])}|P|2.5.1'
    hl7_message.append(msh_segment)
    
    # Create PID (Patient Identification) Segment
    pid_segment = f'PID|||{output_row["Patient MRN"]}^^^^TWSMAMRN||{output_row["LastName"]}^{output_row["FirstName"]}^{output_row["MiddleName"]}||{output_row["DateOfBirth"].replace("-", "")}|{output_row["Sex"]}|||{output_row["APM_Patient_Street1"]}^^{output_row["APM_Patient_City"]}^{output_row["APM_Patient_State"]}^{output_row["APM_Patient_Zip_Code"]}'
    hl7_message.append(pid_segment)
    
    # Create OBR (Observation Request) Segment
    obr_segment = f'OBR|1||||||{convert_datetime(output_row["PerformedDTTM"])}'
    hl7_message.append(obr_segment)
    
    # Create OBX (Observation Result) Segments
    obx_counter = 1
    for _, vital_row in vitals_df.iterrows():
        # Matching the Source Example or HL7 Field to the relevant field in output.csv
        metric = vital_row['Source Example']
        if metric in output_row.values:  # Check if the metric exists in the output row
            try:
                obx_segment = f'OBX|{obx_counter}||{vital_row["HL7 example"]}^LN||{output_row[metric]}|{vital_row["Description"]}'
                hl7_message.append(obx_segment)
                obx_counter += 1
            except KeyError as e:
                print(f"KeyError: {e}. Skipping this metric.")
    
    # Return the combined HL7 message as a string
    return '\n'.join(hl7_message)

# Iterate through each row in the output CSV to create HL7 messages
hl7_messages = []
for _, row in output_df.iterrows():
    hl7_message = create_hl7_message(row, vitals_df)
    hl7_messages.append(hl7_message)

# Write HL7-like messages to a text file
with open('output.hl7.txt', 'w') as f:
    for message in hl7_messages:
        f.write(message + '\n\n')  # Separate each message with a double newline

# Display completion message
print("HL7-like messages have been written to 'output.hl7.txt'.")


Output CSV Columns: Index(['PatientId', 'Patient MRN', 'LastName', 'FirstName', 'MiddleName',
       'DateOfBirth', 'Sex', 'Metric_Desc', 'Metric_Abbr', 'Result',
       'UnitOfMeasure', 'CreateDTTM', 'PerformedDTTM', 'DecodedValue',
       'QOClassificationDE', 'AnswerDE', 'VitalValue', 'NormalizedValue',
       'NumericFinding', 'Unit_Abbr', 'RecordedDTTM', 'QODE', 'Vital_Status',
       'Mode_Name', 'ActivityHeaderID', 'ItemID', 'LoincLabCode',
       'APM_Patient_ID', 'APM_MRN', 'APM_Patient_First_Name',
       'APM_Patient_Last_Name', 'APM_Patient_MI', 'APM_Patient_Street1',
       'APM_Patient_Street2', 'APM_Patient_City', 'APM_Patient_State',
       'APM_Patient_Zip_Code', 'APM_Patient_Country', 'APM_Patient_Sex',
       'APM_Patient_DOB', 'Patient_Home_Phone', 'Patient_Work_Phone',
       'Patient_Work_Ext', 'Patient_Cell_Phone', 'Patient_Primary_Phone_Type',
       'Patient_Primary_Phone_Number', 'Patient_Emailx'],
      dtype='object')
Vitals Description CSV Columns: Index(['

In [13]:
import pandas as pd

# Load the CSV file
csv_file = 'output.csv'  # Replace with your actual CSV file path
df = pd.read_csv(csv_file)

# Print the column names to verify
print(df.columns)


Index(['PatientId', 'Patient MRN', 'LastName', 'FirstName', 'MiddleName',
       'DateOfBirth', 'Sex', 'Metric_Desc', 'Metric_Abbr', 'Result',
       'UnitOfMeasure', 'CreateDTTM', 'PerformedDTTM', 'DecodedValue',
       'QOClassificationDE', 'AnswerDE', 'VitalValue', 'NormalizedValue',
       'NumericFinding', 'Unit_Abbr', 'RecordedDTTM', 'QODE', 'Vital_Status',
       'Mode_Name', 'ActivityHeaderID', 'ItemID', 'LoincLabCode',
       'APM_Patient_ID', 'APM_MRN', 'APM_Patient_First_Name',
       'APM_Patient_Last_Name', 'APM_Patient_MI', 'APM_Patient_Street1',
       'APM_Patient_Street2', 'APM_Patient_City', 'APM_Patient_State',
       'APM_Patient_Zip_Code', 'APM_Patient_Country', 'APM_Patient_Sex',
       'APM_Patient_DOB', 'Patient_Home_Phone', 'Patient_Work_Phone',
       'Patient_Work_Ext', 'Patient_Cell_Phone', 'Patient_Primary_Phone_Type',
       'Patient_Primary_Phone_Number', 'Patient_Emailx'],
      dtype='object')


In [14]:
import hl7
import pandas as pd
from datetime import datetime

# Function to convert date to HL7 format
def convert_datetime(dt_str):
    try:
        return datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')
    except Exception:
        return dt_str

# Load the CSV file
csv_file = 'output.csv'  # Replace with your actual CSV file path
df = pd.read_csv(csv_file)

# Function to create an HL7-like message using python-hl7
def create_hl7_message(row):
    # MSH (Message Header) Segment
    msh_segment = hl7.Segment()
    msh_segment.append("MSH")
    msh_segment.append("^~\\&")
    msh_segment.append("TouchWorks")
    msh_segment.append("Southwest Medical Associates")
    msh_segment.append("Rhapsody^Rhapsody")
    msh_segment.append("Epic^Epic")
    msh_segment.append(convert_datetime(row["PerformedDTTM"]))  # Convert date to HL7 format
    msh_segment.append("")
    msh_segment.append("ORU^R01")
    msh_segment.append(f'{row["Patient MRN"]}_{row["ActivityHeaderID"]}_{row["Result"]}')  # Patient MRN, ActivityHeaderID, and Result
    msh_segment.append("P")
    msh_segment.append("2.5.1")
    
    # PID (Patient Identification) Segment
    pid_segment = hl7.Segment()
    pid_segment.append("PID")
    pid_segment.append("")
    pid_segment.append("")
    pid_segment.append(f'{row["LastName"]}^^^^TWSMAMRN')  # Last name and Medical Record Number
    pid_segment.append(f'{row["FirstName"]}^{row["MiddleName"]}^{row["DateOfBirth"]}')  # First name, Middle name, and Date of Birth
    pid_segment.append("")
    pid_segment.append(f'{row["Sex"]}|{row["Metric_Desc"]}')  # Sex and Metric Description
    pid_segment.append("")
    pid_segment.append(f'{row["APM_Patient_Street1"]}^^{row["APM_Patient_City"]}^{row["APM_Patient_State"]}^{row["APM_Patient_Zip_Code"]}^{row["APM_Patient_Country"]}')  # Address
    
    # OBR (Observation Request) Segment
    obr_segment = hl7.Segment()
    obr_segment.append("OBR")
    obr_segment.append("1")
    obr_segment.append("")
    obr_segment.append("")
    obr_segment.append("")
    obr_segment.append("")
    obr_segment.append("")
    obr_segment.append(f'{row["Result"]}')  # Observation Result
    
    # OBX (Observation/Result) Segment
    obx_segment = hl7.Segment()
    obx_segment.append("OBX")
    obx_segment.append("1")
    obx_segment.append("")
    obx_segment.append(f'{row["LoincLabCode"]}^{row["Metric_Abbr"]}^LN')  # LOINC Code and Metric Abbreviation
    obx_segment.append("")
    obx_segment.append(f'{row["Result"]}')  # Result Value
    obx_segment.append(f'{row["UnitOfMeasure"]}')  # Unit of Measure
    
    # Combine segments into a single HL7 message
    message = hl7.Message()
    message.append(msh_segment)
    message.append(pid_segment)
    message.append(obr_segment)
    message.append(obx_segment)
    
    return message

# Create HL7 messages from the CSV data
hl7_messages = []
for _, row in df.iterrows():
    hl7_message = create_hl7_message(row)
    hl7_messages.append(str(hl7_message))

# Save the HL7-like messages to a text file
with open('output_hl7.txt', 'w') as f:
    for message in hl7_messages:
        f.write(message + '\n\n')

print("HL7 messages have been written to 'output_hl7.txt'.")


HL7 messages have been written to 'output_hl7.txt'.
